In [1]:
import os
import io
import json
import markdown
import warnings
import pandas as pd
import gradio as gr
import IPython.display
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.chains import SequentialChain
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [2]:
warnings.filterwarnings('ignore')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

In [3]:
llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=0.0)

## LLM Chain

It doesn't need history for completion. Therefore, is token-efficiente and it doesn't require the memory of GPT.

In [4]:
load_template_file_prompt = ChatPromptTemplate.from_template("""

You will be provided with a table in a markdown format as << INPUT >>.

If there is not a markdown table in the input return an empty JSON object. Otherwise, return a JSON object formatted to look like:

<< FORMATTING >>
{{{{
    "template_metadata": [
        {{{{
            "header": string, \ name of the column. If the input does not contain a header suggest a name for the column based on its data.
            "type": string, \ type of the data column of the markdown file in the input. 
            "sample": \ put a not null samble of the column. This sample should have the most common value which is not null.
            "categorical": bool \ check if the column is categorical (true) or not categorical (false).
            "categories_list": [] \ list of the unique values if the column is categorical.
            "date_format": null except if type is date suggest SQL DATE FORMAT for converting the column values to date.
            "description": string \ descrption of this column based only on its data.
        }}}},
        ...
    ]
}}}}


<< INPUT >>
{input_template}

<< OUTPUT >>
""")

In [5]:
load_file_prompt = ChatPromptTemplate.from_template("""

You will be provided with a table in a markdown format as << INPUT >>.
Also you will provided with the name of the table as << TABLE >>

If there is not a markdown table in the input return an empty JSON object. Otherwise, return a JSON object formatted to look like:

<< FORMATTING >>
{{{{ 
    "table_name": string, \ name of the table. you can find it at the begining of the input.
    "file_metadata": [
        {{{{
            "header": string, \ name of the column. If the input does not contain a header suggest a name for the column based on its data.
            "type": string, \ type of the data column of the markdown file in the input. 
            "sample": \ put a not null samble of the column. This sample should have the most common value which is not null.
            "date_format": string, \ null except if type is date suggest SQL DATE FORMAT for converting the column values to date.
            "description": string \ description of this column based only on its data.
        }}}},
        ...
    ],
    "table" : string \ the complete markdown table in the input
}}}}

<< TABLE >>
{table_name}

<< INPUT >>
{input_file}

""")

In [6]:
formating_header_prompt = ChatPromptTemplate.from_template("""
You will receive two JSON Objects called table_info and template_description as inputs << INPUT >.

Follow the following instruction:

    Step 1: Go over the list table_info['file_metadata'] and find what is the N header in the list template_description['template_metadata'] most similar to the "new_file" table header.
    Step 2: return the same "table_info" JSON object adding the following information:

"header_match": [
        {{{{
            "template_header": string, \ header of "template" table
            "table_header": string, \ header of "new_file" table table most similar to the N header of "template" table
        }}}},
        ...
    ],

"template_header" is the N header of "template" table most similar to the correspoding header of "new_file" table. Determine this similarity based only on the metadata such as:
    * Data types
    * Samples of both tables
    * Description

<< INPUT >>
{table_info}
{template_description}

<< OUTPUT >>
""")

In [7]:
table_proposal_prompt = ChatPromptTemplate.from_template("""
You will receive two JSON Objects called table_header_match and template_description as inputs << INPUT >.

Follow the next instructions for generating a markdown table:
    
    Step 1: Go over the list table_header_match["header_match"].
    Step 2: For each one of the table_header_match["header_match"]["table_header"], replace the header of the markdown table in table_header_match["table"] by table_header_match["header_match"]["template_header"]
    Step 3: The new markdown table must have only the columns in listed in table_header_match["header_match"]["template_header"]. Remove all the remaining columns different to table_header_match["header_match"]["template_header"].

Return the new markdown table in a JSON Object formatted to look like:

<< FORMATTING >>
{{{{ 
    "table_name": string, \ name of the table. you can find it in table_header_match["table_name"].
    "file_metadata": [
        {{{{
            "header": string, \ name of the column.
            "type": string, \ type of the data column of the markdown file in the input. 
            "sample": \ put a not null samble of the column. This sample should have the most common value which is not null.
            "categorical": bool \ check if the column is categorical (true) or not categorical (false).
            "categories_list": [] \ list of the unique values if the column is categorical.
            "date_format": null except if type is date suggest SQL DATE FORMAT for converting the column values to date.
            "description": string \ descrption of this column based only on its data.
        }}}},
        ...
    ],
    "modified_table" : string, \ the new markdown table.
    "template_metadata": template_description["template_metadata"]  \ the template metadata.
}}}}

<< INPUT >>
{table_header_match}
{template_description}

""")

In [8]:
formating_categories_prompt = ChatPromptTemplate.from_template("""
You will receive a JSON Object called simple_table as input << INPUT >.

Based on the given input, the task is to find the column in the "modified_table" that is most similar to the N header in the "template_metadata" list. Then, select only the categorical columns and return the "simple_table" JSON object with the added information.

Follow the next instructions for generating a markdown table:
Step 1: compare the headers in the "modified_table" with the headers in the "template_metadata" list. We will  iterate over the columns in the "modified_table" and find the column that is most similar to the N header in the "template_metadata" list.
Step 2: After finding the most similar column, check if it is a categorical column by checking the "categorical" key in the "file_metadata" list.
Step 3: Add the following information to the "simple_table" JSON object.
Step 4: Return the updated  "simple_table" JSON object.

"categories_match": [
    {{{{
        "categories_list": string, \ list of categories in simple_table['template_metadata']
        "table_header": string, \ header of markdown table in simple_table["modified_table"] most similar to the N header of "template_metadata"
    }}}},
    ...
],

<< INPUT >>
{simple_table}

<< OUTPUT >>
Return the updated  "simple_table" JSON object.
""")

In [9]:
categories_result_prompt = ChatPromptTemplate.from_template("""
You will receive a JSON Object called table_categories_match as input << INPUT >.

Based on the given input, the task is to generate a markdown table by replacing each value in the categorical columns of the "modified_table" with the most similar item from the "categories_list" in the "categories_match" section. The updated table should be returned as a JSON object.

Follow the next instructions for generating a markdown table:
Step 1: Iterate over each categorical column in the "modified_table".
Step 2: Replace each value in the column with the most similar item from the "categories_list" in the "categories_match" section.
Setp 3: the new markdown table will be called correct_cats_markdown_table

Return the new markdown table (correct_cats_markdown_table) in a JSON Object formatted to look like:

<< FORMATTING >>
{{{{ 
    "table_name": string, \ name of the table. you can find it in table_categories_match["table_name"].
    "file_metadata": [
    {{{{
        "header": string, \ name of the column.
        "type": string, \ type of the data column of the markdown file in the input. 
        "sample": \ put a not null samble of the column. This sample should have the most common value which is not null.
        "categorical": bool \ check if the column is categorical (true) or not categorical (false).
        "categories_list": [] \ list of the unique values if the column is categorical.
        "date_format": null except if type is date suggest SQL DATE FORMAT for converting the column values to date.
        "description": string \ descrption of this column based only on its data.
    }}}},
    ...
    ],
    "table" : string, \ the new markdown table (correct_cats_markdown_table).
    "template_metadata": table_categories_match["template_metadata"]  \ the template metadata.
}}}}

<< INPUT >>
{table_categories_match}

<< OUTPUT >>
Return only the JSON Object

""")

In [10]:
formating_dates_prompt = ChatPromptTemplate.from_template("""
You will receive a JSON Object called table_categories_result as input << INPUT >.

Change the format of each one the rows of date columns in the markdown in table_categories_result["table"] according to the date format in the list table_categories_result["template_metadata"]

The new markdown table will be called correct_dates_markdown_table

Return the new markdown table (correct_dates_markdown_table) in a JSON Object formatted to look like:

<< FORMATTING >>
{{{{ 
    "table_name": string, \ name of the table. you can find it in table_categories_result["table_name"].
    "table" : string, \ the new markdown table (correct_dates_markdown_table).
    "template_metadata": table_categories_result["template_metadata"]  \ the template metadata.
}}}}

<< INPUT >>
{table_categories_result}

<< OUTPUT >>
Return only the JSON Object

""")

In [11]:
formating_strings_prompt = ChatPromptTemplate.from_template("""
You will receive a JSON Object called table_dates_result as input << INPUT >.

Based on the given input, the task is to find the column in the markdown "table" that is most similar to the N header in the "template_metadata" list. Then, select only the string columns and return the "table_dates_result" JSON object with the added information.

Follow the next instructions for generating a markdown table:
Step 1: compare the headers in the "table" with the headers in the "template_metadata" list. We will  iterate over the columns in the "table" and find the column that is most similar to the N header in the "template_metadata" list.
Step 2: After finding the most similar column, check if it is a string column by checking the "type" key in the "file_metadata" list.
Step 3: Ignore if it is a categorical, numerical or date column by checking the "categorical" key in the "file_metadata" list.
Step 4: Add the following information to the "table_dates_result" JSON object.
Step 5: Return the updated  "table_dates_result" JSON object.

"strings_match": [
    {{{{
        "selected_sample": string, \ sample of data in table_dates_result["template_metadata"]
        "table_header": string, \ header of markdown table in table_dates_result["table"] most similar to the N header of "template_metadata"
    }}}},
    ...
],

<< INPUT >>
{table_dates_result}

<< OUTPUT >>
Return the updated "table_dates_result" JSON object.
""")

In [12]:
chain_template_load = LLMChain(llm=llm, prompt=load_template_file_prompt, 
                     output_key="template_description"
                    )
chain_load = LLMChain(llm=llm, prompt=load_file_prompt, 
                     output_key="table_info"
                    )
chain_header_formatting = LLMChain(llm=llm, prompt=formating_header_prompt, 
                     output_key="table_header_match"
                    )
chain_proposal = LLMChain(llm=llm, prompt=table_proposal_prompt, 
                     output_key="simple_table"
                    )
chain_cats_formatting = LLMChain(llm=llm, prompt=formating_categories_prompt, 
                     output_key="table_categories_match"
                    )
chain_cats_result = LLMChain(llm=llm, prompt=categories_result_prompt, 
                     output_key="table_categories_result"
                    )
chain_dates_result = LLMChain(llm=llm, prompt=formating_dates_prompt, 
                     output_key="table_dates_result"
                    )
chain_strings_formatting = LLMChain(llm=llm, prompt=formating_strings_prompt, 
                     output_key="table_strings_match"
                    )

In [13]:
overall_chain = SequentialChain(
    chains=[chain_template_load, chain_load, chain_header_formatting, chain_proposal, chain_cats_formatting, chain_cats_result, chain_dates_result, chain_strings_formatting],
    input_variables=["input_template","table_name","input_file"],
    output_variables=["table_header_match","table_categories_match","table_strings_match"],
    verbose=True
)

## User Interface

There is a chatbot, here GPT memory handle the token usage.

In [14]:
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm, 
    verbose=False,
    memory=memory
)

In [15]:
def markdown_to_html(md_table_string):
    return markdown.markdown(md_table_string, extensions=['markdown.extensions.tables'])

In [16]:
def process_csv(file, file_label):
    global _file_buffer
    
    # Read the uploaded CSV file with pandas
    df = pd.read_csv(io.StringIO(file.decode('utf-8')))
    
    # Convert the DataFrame to an HTML table with added styles
    html_table = df.to_html(classes='table table-striped')
    
    # Add CSS for scrollable table
    styled_table = f"""
    <div style="max-width: 100%; overflow-x: auto;">
        {html_table}
    </div>
    """
    _file_buffer[file_label] = df.to_markdown()
    
    return styled_table

def process_template(file):
    return process_csv(file, 'template')

def process_new_file(file):
    return process_csv(file, 'new_file')

In [17]:
def format_chat_prompt(message, chat_history, instruction):
    prompt = f"System Context:{instruction}"
    # for turn in chat_history:
    #     user_message, bot_message = turn
    #     prompt = f"{prompt}\nUser: {user_message}\nAssistant: {bot_message}"
    # prompt = f"{prompt}\nUser: {message}\nAssistant:"
    return prompt

In [18]:
def respond(message, chat_history, instruction, temperature=0.7):
    formatted_prompt = format_chat_prompt(message, chat_history, instruction)
    bot_message = conversation.predict(input=formatted_prompt)
    chat_history.append((message, bot_message))
    return "", chat_history

In [19]:
def tables_analysis():
    global _file_buffer
    execution_chain = overall_chain({
        "input_template":_file_buffer['template'],
        "table_name":"new_file",
        "input_file":_file_buffer['new_file']
    })
    result_chain = json.loads(execution_chain['table_strings_match'])
    show_table_html = markdown_to_html(result_chain['table'])
    system_context = f"""
This is the transformed data according to the template.

Transformed Data:

{result_chain['table']}

Template:

{execution_chain['input_template']}

Input File:

{execution_chain['input_file']}
    """
    return show_table_html, system_context
    

In [20]:
_file_buffer = {
    'template':'',
    'new_file':'',
}

In [21]:
with gr.Blocks() as demo:
    
    with gr.Row():
    
        with gr.Column():
            # Load Data
            gr.HTML('<h1 align="center">Step 1: Load Data</h1>')
            
            upload_template = gr.inputs.File(type="bytes", label="Upload Template")
            data_template = gr.outputs.HTML(label="Template")
            upload_template.upload(process_template, inputs=upload_template, outputs=data_template)
            
            upload_file = gr.inputs.File(type="bytes", label="Upload New File")
            data_file = gr.outputs.HTML(label="New File")
            upload_file.upload(process_new_file, inputs=upload_file, outputs=data_file)

        with gr.Column():
            # Analyse Data
            gr.HTML('<h1 align="center">Step 2: Analysis </h1>')
            gr.HTML('<h2 align="left">Analysis of Table</h2>')
            gr.HTML('<h3 align="left">This process could take up to 5 minutes.</h3>')
            btn_analyse = gr.Button("Analyse Table")
            data_proposal = gr.outputs.HTML(label="Proposal")
            
            # Chatbot
            gr.HTML('<h1 align="center">Step 3: Chatbot </h1>')
            chatbot = gr.Chatbot(height=446, label='Chatbot') #just to fit the notebook
            msg = gr.Textbox(label="Prompt")
            with gr.Accordion(label="Advanced options",open=False):
                system_context = gr.Textbox(label="System Context", lines=2, value="A conversation between a user and an LLM-based AI assistant. The assistant gives helpful and honest answers.")
                temperature = gr.Slider(label="temperature", minimum=0.1, maximum=1, value=0.7, step=0.1)
            btn = gr.Button("Submit")
            clear = gr.ClearButton(components=[msg, chatbot], value="Clear console")
            
            # Actions
            btn_analyse.click(tables_analysis, inputs=None, outputs=[data_proposal,system_context])
            btn.click(respond, inputs=[msg, chatbot, system_context], outputs=[msg, chatbot])
            msg.submit(respond, inputs=[msg, chatbot, system_context], outputs=[msg, chatbot]) #Press enter to submit

gr.close_all()
demo.queue().launch(share=False, server_port=int(os.environ['GRADIO_SERVER_PORT']))


Running on local URL:  http://127.0.0.1:5150

To create a public link, set `share=True` in `launch()`.
